In [3]:
# https://github.com/dragen1860/TensorFlow-2.x-Tutorials/tree/master/01-TF2.0-Overview

In [4]:
# Fully-connected Network
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

2021-09-10 18:43:39.351065: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [5]:
# Load numpy data

import numpy as np
import tensorflow as tf

DATA_URL = 'https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'

path = tf.keras.utils.get_file('mnist.npz', DATA_URL)
with np.load(path) as data:
    train_examples = data['x_train']
    train_labels = data['y_train']
    test_examples = data['x_test']
    test_labels = data['y_test']

11493376/11490434 [==============================] - 0s 0us/step


In [6]:
print(type(train_examples), train_examples.shape)
print(type(test_examples), test_examples.shape)
print(type(train_labels), train_labels.shape)
print(type(test_labels), test_labels.shape)

<class 'numpy.ndarray'> (60000, 28, 28)
<class 'numpy.ndarray'> (10000, 28, 28)
<class 'numpy.ndarray'> (60000,)
<class 'numpy.ndarray'> (10000,)


In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_examples, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_examples, test_labels))

2021-09-10 18:43:48.691975: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-09-10 18:43:48.706094: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-09-10 18:43:48.759951: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-10 18:43:48.761021: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-09-10 18:43:48.761073: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-09-10 18:43:48.788504: I tensorflow/stream_executor/platform/def

In [8]:
# Shuffle and batch the datasets
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [9]:
def mnist_dataset():
    (x, y), _ = datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    # https://www.gcptutorials.com/article/how-to-use-map-function-with-tensorflow-datasets
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    # https://www.tensorflow.org/api_docs/python/tf/cast
    return x, y

In [10]:
model = keras.Sequential([
    layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    layers.Dense(100, activation='relu'),
    layers.Dense(100, activation='relu'),
    layers.Dense(10)])

optimizer = optimizers.Adam()

With our data and model set up, we can start setting up the training procedure. For the methods here, we use the @tf.function AutoGraph decorator to pre-compile our methods as TensorFlow computational graphs. TensorFlow 2.0 is fully imperative, so the AutoGraph decorator isn't necessary for our code to work, but it speeds up execution and lets us take advantage of graph execution, so @tf.function is definitely worth using in our case.

In [11]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):

    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

  # compute gradient
    grads = tape.gradient(loss, model.trainable_variables)
    # https://www.tensorflow.org/api_docs/python/tf/GradientTape
  # update to weights
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
  # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Optimizer

    accuracy = compute_accuracy(logits, y)

  # loss and accuracy is scalar tensor
    return loss, accuracy

def train(epoch, model, optimizer):

    train_ds = mnist_dataset()
    loss = 0.0
    accuracy = 0.0
    for step, (x, y) in enumerate(train_ds):
        loss, accuracy = train_one_step(model, optimizer, x, y)

        if step % 500 == 0:
            print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

    return loss, accuracy

Now that we have our training procedure set up, we can throw it in a loop and start training!

In [12]:
for epoch in range(20):
    loss, accuracy = train(epoch, model, optimizer)

print('Final epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())

2021-09-10 18:43:52.474113: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-10 18:43:52.479424: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz
2021-09-10 18:43:53.812139: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-09-10 18:43:54.734546: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11


epoch 0 : loss 2.3214438 ; accuracy 0.06
epoch 1 : loss 0.26186752 ; accuracy 0.93
epoch 2 : loss 0.1625739 ; accuracy 0.94
epoch 3 : loss 0.060843468 ; accuracy 0.99
epoch 4 : loss 0.07338034 ; accuracy 0.98
epoch 5 : loss 0.017805804 ; accuracy 1.0
epoch 6 : loss 0.018908933 ; accuracy 1.0
epoch 7 : loss 0.019537352 ; accuracy 1.0
epoch 8 : loss 0.028114202 ; accuracy 1.0
epoch 9 : loss 0.015089001 ; accuracy 1.0
epoch 10 : loss 0.028484225 ; accuracy 1.0
epoch 11 : loss 0.054152954 ; accuracy 0.99
epoch 12 : loss 0.006558891 ; accuracy 1.0
epoch 13 : loss 0.00673928 ; accuracy 1.0
epoch 14 : loss 0.008107851 ; accuracy 1.0
epoch 15 : loss 0.011208303 ; accuracy 1.0
epoch 16 : loss 0.0044960077 ; accuracy 1.0
epoch 17 : loss 0.0022714527 ; accuracy 1.0
epoch 18 : loss 0.012544717 ; accuracy 0.99
epoch 19 : loss 0.002500914 ; accuracy 1.0
Final epoch 19 : loss 0.028578045 ; accuracy 0.99


## Convolutional Network

Now that we've gotten our feet we with a simple DNN, let's try something more advanced. Although the process is the same, we'll be working with some additional features:

- Convolution, pooling, and dropout layers for building more complex models
- Visualizing training with TensorBoard
- Validation and test set evaluation for measuring generalizability
- Exporting with SavedModel to save training progress and deploy trained models

As usual, we'll start by preparing our MNIST data.

In [13]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers, metrics

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

In [14]:
def mnist_datasets():
    (x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
    # Numpy defaults to dtype=float64; TF defaults to float32. Stick with float32.
    x_train, x_test = x_train / np.float32(255), x_test / np.float32(255)
    y_train, y_test = y_train.astype(np.int64), y_test.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    return train_dataset, test_dataset


train_ds, test_ds = mnist_datasets()
train_ds = train_ds.shuffle(60000).batch(100)
test_ds = test_ds.batch(100)

In [15]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.BatchDataset

Now, let's use use Conv2D, MaxPooling2D, and Dropout from keras.layers to build a keras.Sequential convolutional model.

In [16]:
model = tf.keras.Sequential([
    layers.Reshape(
        target_shape=[28, 28, 1],
        input_shape=(28, 28,)),
    layers.Conv2D(2, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Conv2D(4, 5, padding='same', activation=tf.nn.relu),
    layers.MaxPooling2D((2, 2), (2, 2), padding='same'),
    layers.Flatten(),
    layers.Dense(32, activation=tf.nn.relu),
    layers.Dropout(rate=0.4),
    layers.Dense(10)])

optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.5)

Next, let's set up forward and backward functionality. In addition to the training procedure, we'll also write test() a method for evaluation, and use **tf.python.summary_ops_v2** to record training summaries to TensorBoard.

In [17]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

def train_step(model, optimizer, images, labels):

    # Record the operations used to compute the loss, so that the gradient
    # of the loss with respect to the variables can be computed.
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss = compute_loss(labels, logits)
        compute_accuracy(labels, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

def train(model, optimizer, dataset, log_freq=50):
    """
    Trains model on `dataset` using `optimizer`.
    """
    # Metrics are stateful. They accumulate values and return a cumulative
    # result when you call .result(). Clear accumulated values with .reset_states()
    avg_loss = metrics.Mean('loss', dtype=tf.float32)

    # Datasets can be iterated over like any other Python iterable.
    for images, labels in dataset:
        loss = train_step(model, optimizer, images, labels)
        avg_loss(loss)

        if tf.equal(optimizer.iterations % log_freq, 0):
            # summary_ops_v2.scalar('loss', avg_loss.result(), step=optimizer.iterations)
            # summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=optimizer.iterations)
            print('step:', int(optimizer.iterations),
                  'loss:', avg_loss.result().numpy(),
                  'acc:', compute_accuracy.result().numpy())
            avg_loss.reset_states()
            compute_accuracy.reset_states()

def test(model, dataset, step_num):
    """
    Perform an evaluation of `model` on the examples from `dataset`.
    """
    avg_loss = metrics.Mean('loss', dtype=tf.float32)

    for (images, labels) in dataset:
        logits = model(images, training=False)
        avg_loss(compute_loss(labels, logits))
        compute_accuracy(labels, logits)

    print('Model test set loss: {:0.4f} accuracy: {:0.2f}%'.format(
        avg_loss.result(), compute_accuracy.result() * 100))

    print('loss:', avg_loss.result(), 'acc:', compute_accuracy.result())
    # summary_ops_v2.scalar('loss', avg_loss.result(), step=step_num)
    # summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=step_num)

Now that we have our data, model, and training procedure ready, we just need to designate a directory and create a **tf.train.Checkpoint** file to save our parameters to as we train.

In [20]:
# Where to save checkpoints, tensorboard summaries, etc.
MODEL_DIR = '../kaggle/working/'


def apply_clean():
    if tf.io.gfile.exists(MODEL_DIR):
        print('Removing existing model dir: {}'.format(MODEL_DIR))
        tf.io.gfile.rmtree(MODEL_DIR)


apply_clean()

checkpoint_dir = os.path.join(MODEL_DIR, 'checkpoints')
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')

checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)

# Restore variables on creation if a checkpoint exists.
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

We're finally ready to start training!

In [22]:
NUM_TRAIN_EPOCHS = 5

for i in range(NUM_TRAIN_EPOCHS):
    start = time.time()
    #   with train_summary_writer.as_default():
    train(model, optimizer, train_ds, log_freq=500)
    end = time.time()
    print('Train time for epoch #{} ({} total steps): {}'.format(
        i + 1, int(optimizer.iterations), end - start))
    #   with test_summary_writer.as_default():
    #     test(model, test_ds, optimizer.iterations)
    checkpoint.save(checkpoint_prefix)
    print('saved checkpoint.')

export_path = os.path.join(MODEL_DIR, 'export')
tf.saved_model.save(model, export_path)
print('saved SavedModel for exporting.')

2021-09-10 18:51:17.678750: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


step: 500 loss: 1.9935889 acc: 0.29872
Train time for epoch #1 (600 total steps): 16.93648338317871
saved checkpoint.
step: 1000 loss: 0.7801143 acc: 0.72106
Train time for epoch #2 (1200 total steps): 11.061720609664917
saved checkpoint.
step: 1500 loss: 0.566319 acc: 0.80318
Train time for epoch #3 (1800 total steps): 11.447207927703857
saved checkpoint.
step: 2000 loss: 0.43725118 acc: 0.85012
Train time for epoch #4 (2400 total steps): 11.01601529121399
saved checkpoint.
step: 2500 loss: 0.36324552 acc: 0.8743
step: 3000 loss: 0.34172356 acc: 0.89308
Train time for epoch #5 (3000 total steps): 11.19942331314087
saved checkpoint.


2021-09-10 18:52:19.543997: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


saved SavedModel for exporting.
